## 实验题目5 高斯(Gauss)列主元消去法

### 代码实现

In [51]:
using Printf
using LinearAlgebra

In [52]:
# from: https://stackoverflow.com/questions/58667332/is-there-a-way-to-swap-columns-in-o1-in-julia
function swapcols!(X::AbstractMatrix, i::Integer, j::Integer)
    @inbounds for k = 1:size(X, 1)
        X[k, i], X[k, j] = X[k, j], X[k, i]
    end
end
# from: https://discourse.julialang.org/t/swap-cols-rows-of-a-matrix/47904/9
function _swapcol!(x, i, j)
    for k in axes(x, 1)  # <- give dimension as input to axes function
        x[k, i], x[k, j] = x[k, j], x[k, i]
    end
end

_swapcol! (generic function with 1 method)

In [53]:
function swaprows!(X::AbstractMatrix, i::Integer, j::Integer)
    @inbounds for k = 1:size(X, 2)
        X[i, k], X[j, k] = X[j, k], X[i, k]
    end
end

swaprows! (generic function with 1 method)

In [54]:
# https://stackoverflow.com/questions/45396685/what-does-an-exclamation-mark-mean-after-the-name-of-a-function
# https://people.richland.edu/james/lecture/m116/matrices/pivot.html
function pivoting!(A::Matrix{Float64}, k::Integer, n::Integer)
    val, idx = findmax(A[k:n, k])
    idx += k - 1  # index must add previous length that omitted by slice operator
    return val, idx
end
function pivoting!(A::Matrix{Float64}, b::Vector{Float64}, k::Integer, n::Integer, implicit::Bool)
    s = [maximum(A[i, k:n]) for i in k:n]
    if 0 in s
        println("Cannot solve a singular matrix!")
        return
    end
    if implicit
        val, idx = findmax(A[k:n, k] ./ s[1:n-k+1])
    else
        A[k:n, k:n] = A[k:n, k:n] ./ s
        b[k:n] = b[k:n] ./ s
        val, idx = findmax(A[k:n, k])
    end
    idx += k - 1  # index must add previous length that omitted by slice operator
    return val, idx
end

pivoting! (generic function with 2 methods)

In [55]:
# Gauss列主元消去法
# Todo: modify it using . operator
function gauss(n, A::Matrix{Float64}, b::Vector{Float64})
    for k = 1:n-1
        # select pivot in columns
        val, idx = pivoting!(A, k, n)
        if val == 0
            println("Cannot solve a singular matrix!")
            return
        end
        # swap rows
        if idx != k
            swaprows!(A, idx, k)
            b[idx], b[k] = b[k], b[idx]
        end
        # elimination
        for i = k+1:n
            m = A[i, k] / A[k, k]
            A[i, :] -= A[k, :] * m
            b[i] -= b[k] * m
        end
    end
    if A[n, n] == 0
        println("Cannot solve a singular matrix!")
        return
    end
    # https://stackoverflow.com/questions/62142717/julia-quick-way-to-initialise-an-empty-array-thats-the-same-size-as-another
    x = similar(b, Float64)
    x[n] = b[n] / A[n, n]
    for k = n-1:-1:1  # the usage of reverse sequence
        x[k] = (b[k] - dot(A[k, k+1:n], x[k+1:n])) / A[k, k]  # something really annoying 
    end
    x
end


gauss (generic function with 2 methods)

In [56]:
# Gauss列主元消去法
# Todo: modify it using . operator
function gauss(n, A::Matrix{Float64}, b::Vector{Float64}, implicit::Bool)
    for k = 1:n-1
        # select pivot in columns
        val, idx = pivoting!(A, b, k, n, implicit)
        if val == 0
            println("Cannot solve a singular matrix!")
            return
        end
        # swap rows
        if idx != k
            swaprows!(A, idx, k)
            b[idx], b[k] = b[k], b[idx]
        end
        # elimination
        for i = k+1:n
            m = A[i, k] / A[k, k]
            A[i, :] -= A[k, :] * m
            b[i] -= b[k] * m
        end
    end
    if A[n, n] == 0
        println("Cannot solve a singular matrix!")
        return
    end
    # https://stackoverflow.com/questions/62142717/julia-quick-way-to-initialise-an-empty-array-thats-the-same-size-as-another
    x = similar(b, Float64)
    x[n] = b[n] / A[n, n]
    for k = n-1:-1:1  # the usage of reverse sequence
        x[k] = (b[k] - dot(A[k, k+1:n], x[k+1:n])) / A[k, k]  # something really annoying 
    end
    x
end


gauss (generic function with 2 methods)

### 测试代码

#### Test 1 - Correctness

In [57]:
# test random result of standard library 
# test pass
for i in 1:5
    M = rand(300, 300)
    v = rand(300)
    A, b = copy(M), copy(v)  # ? Todo: move this line into try block will extend compilation time
    try
        @time print("$(norm(A \ b - gauss(size(A, 1), A, b), 2))\t")

        A, b = copy(M), copy(v)
        @time print("$(norm(A \ b - gauss(size(A, 1), A, b, false), 2))\t")  # implicit=false

        A, b = copy(M), copy(v)
        @time print("$(norm(A \ b - gauss(size(A, 1), A, b, true), 2))\t")  # implicit=true
    catch SigularException
        println("Cannot solve a singular matrix!")
    end
    println()
end

7.131336177052355e-12	  0.265213 seconds (278.71 k allocations: 444.834 MiB, 12.18% gc time, 35.44% compilation time)
3.675108930920388e-12	  0.527333 seconds (475.89 k allocations: 665.127 MiB, 9.91% gc time, 43.12% compilation time)
3.3255467186003486e-12	  0.209261 seconds (227.26 k allocations: 515.581 MiB, 15.77% gc time)

1.9261226059524225e-12	  0.159760 seconds (180.32 k allocations: 439.810 MiB, 17.96% gc time)
1.6389808860967015e-12	  0.247379 seconds (228.37 k allocations: 653.650 MiB, 16.19% gc time)
8.861486103067028e-13	  0.211765 seconds (227.26 k allocations: 515.581 MiB, 16.48% gc time)

1.9564591550268644e-10	  0.161792 seconds (180.32 k allocations: 439.810 MiB, 19.98% gc time)
3.278877074521003e-10	  0.248221 seconds (228.37 k allocations: 653.650 MiB, 15.84% gc time)
3.9947877031880073e-10	  0.211226 seconds (227.26 k allocations: 515.581 MiB, 15.70% gc time)

1.8708640396130578e-12	  0.161135 seconds (180.32 k allocations: 439.810 MiB, 18.62% gc time)
2.1881244041

#### Test 2 - Performance 

In [58]:
# test random result of standard library 
# test pass
for i in 1:5
    M = rand(300, 300)
    v = rand(300)
    A, b = copy(M), copy(v)
    try
        @time A \ b

        A, b = copy(M), copy(v)
        @time gauss(size(A, 1), A, b)

        A, b = copy(M), copy(v)
        @time gauss(size(A, 1), A, b, false)  # implicit=false

        A, b = copy(M), copy(v)
        @time gauss(size(A, 1), A, b, true)  # implicit=true
    catch SigularException
        println("Cannot solve a singular matrix!")
    end
    println()
end

  0.006741 seconds (4 allocations: 708.172 KiB)
  0.161966 seconds (180.30 k allocations: 439.115 MiB, 20.40% gc time)
  0.243198 seconds (228.35 k allocations: 652.955 MiB, 14.35% gc time)
  0.215559 seconds (227.24 k allocations: 514.886 MiB, 17.17% gc time)

  0.006149 seconds (4 allocations: 708.172 KiB)
  0.199172 seconds (180.30 k allocations: 439.115 MiB, 19.29% gc time)
  0.239563 seconds (228.35 k allocations: 652.955 MiB, 14.83% gc time)
  0.191920 seconds (227.24 k allocations: 514.886 MiB, 16.51% gc time)

  0.006614 seconds (4 allocations: 708.172 KiB)
  0.151911 seconds (180.30 k allocations: 439.115 MiB, 18.53% gc time)
  0.231331 seconds (228.35 k allocations: 652.955 MiB, 15.73% gc time)
  0.212872 seconds (227.24 k allocations: 514.886 MiB, 18.09% gc time)

  0.006014 seconds (4 allocations: 708.172 KiB)
  0.158211 seconds (180.30 k allocations: 439.115 MiB, 19.00% gc time)
  0.254974 seconds (228.35 k allocations: 652.955 MiB, 14.90% gc time)
  0.238197 seconds (227.

#### Test 3 - Special Matrix

Bug: 处理上三角矩阵计算有异常

Todo: 应当增加对于特殊矩阵的测试，待测试列表同Julia Special Matrix:

- [ ] Symmetric
- [ ] Hermitian
- [ ] UpperTriangular
- [ ] UnitUpperTriangular
- [ ] LowerTriangular
- [ ] UnitLowerTriangular
- [ ] UpperHessenberg
- [ ] Tridiagonal
- [ ] SymTridiagonal
- [ ] Bidiagonal
- [ ] Diagonal
- [ ] UniformScaling

基本语法为 `M = SpecialMatrix(rand(300,300))`

### 实验题目

#### 问题 1

In [59]:
A = [0.4096 0.1234 0.3678 0.2943
    0.2246 0.3872 0.4015 0.1129
    0.3645 0.1920 0.3781 0.0643
    0.1784 0.4002 0.2786 0.3927]
b = [1.1951; 1.1262; 0.9989; 1.2499]
display(@time A \ b)
display(@time gauss(size(A, 1), A, b))
# display(@time gauss(size(A, 1), A, b, false))  # implicit=false
# display(@time gauss(size(A, 1), A, b, true))  # implicit=true

4-element Vector{Float64}:
 0.9999999999999994
 1.0
 1.0000000000000004
 1.0000000000000002

4-element Vector{Float64}:
 1.0000000000000027
 1.0000000000000018
 0.9999999999999971
 0.9999999999999992

  0.000025 seconds (3 allocations: 384 bytes)
  0.000018 seconds (34 allocations: 3.031 KiB)


In [60]:
A = [136.01 90.860 0 0
    90.860 98.810 -67.590 0
    0 -67.590 132.01 46.260
    0 0 46.260 177.17]
b = [226.87; 122.08; 110.68; 223.43]
display(@time A \ b)
display(@time gauss(size(A, 1), A, b))
# display(@time gauss(size(A, 1), A, b, false))  # implicit=false
# display(@time gauss(size(A, 1), A, b, true))  # implicit=true

4-element Vector{Float64}:
 1.0000000000000704
 0.9999999999998946
 0.9999999999999408
 1.0000000000000155

4-element Vector{Float64}:
 1.000000000000133
 0.9999999999998009
 0.999999999999888
 1.0000000000000293

  0.000013 seconds (3 allocations: 384 bytes)
  0.000015 seconds (34 allocations: 3.031 KiB)


In [61]:
A = [1 1/2 1/3 1/4
     1/2 1/3 1/4 1/5
     1/3 1/4 1/5 1/6
     1/4 1/5 1/6 1/7]
b = [25 / 12; 77 / 60; 57 / 60; 319 / 420]
display(@time A \ b)
display(@time gauss(size(A, 1), A, b))
# display(@time gauss(size(A, 1), A, b, false))  # implicit=false
# display(@time gauss(size(A, 1), A, b, true))  # implicit=true

4-element Vector{Float64}:
 0.9999999999999779
 1.000000000000241
 0.9999999999994366
 1.0000000000003588

4-element Vector{Float64}:
 0.9999999999999927
 1.0000000000000688
 0.9999999999998556
 1.0000000000000846

  0.000019 seconds (3 allocations: 384 bytes)
  0.000014 seconds (34 allocations: 3.031 KiB)


In [62]:
A = Float64.([10 7 8 7
    7 5 6 5
    8 6 10 9
    7 5 9 10])
b = Float64.([32; 23; 33; 31])
display(@time A \ b)
display(@time gauss(size(A, 1), A, b))
# display(@time gauss(size(A, 1), A, b, false))  # implicit=false
# display(@time gauss(size(A, 1), A, b, true))  # implicit=true

4-element Vector{Float64}:
 1.000000000000083
 0.9999999999998619
 1.000000000000035
 0.999999999999979

4-element Vector{Float64}:
 1.0
 1.0
 1.0
 1.0

  0.000016 seconds (3 allocations: 384 bytes)
  0.000012 seconds (34 allocations: 3.031 KiB)


#### 问题 2


In [63]:
A = [197 305 -206 -804
     46.8 71.3 -47.4 52.0
     88.6 76.4 -10.8 802
     1.45 5.90 6.13 36.5]
b = [136; 11.7; 25.1; 6.60]
display(@time A \ b)
# display(A * (A \ b))
display(@time gauss(size(A, 1), A, b))
# display(@time gauss(size(A, 1), A, b, false))  # implicit=false
# display(@time gauss(size(A, 1), A, b, true))  # implicit=true

4-element Vector{Float64}:
  0.9536791069017718
  0.32095684552110354
  1.0787080757932384
 -0.09010850953957893

4-element Vector{Float64}:
  0.9536791069017717
  0.3209568455211036
  1.0787080757932384
 -0.09010850953957895

  0.000014 seconds (3 allocations: 384 bytes)
  0.000018 seconds (34 allocations: 3.031 KiB)


In [64]:
A = [0.5398 0.7161 -0.5554 -0.2982
    0.5257 0.6924 0.3565 -0.6255
    0.6465 -0.8187 -0.1872 0.1291
    0.5814 0.9400 -0.7779 -0.4042]
b = [0.2058; -0.0503; 0.1070; 0.1859]
display(@time A \ b)
# display(A * (A \ b))
display(@time gauss(size(A, 1), A, b))
# display(@time gauss(size(A, 1), A, b, false))  # implicit=false
# display(@time gauss(size(A, 1), A, b, true))  # implicit=true

4-element Vector{Float64}:
 0.5161772979585422
 0.4152194728301359
 0.10996610286788958
 1.0365392233362019

4-element Vector{Float64}:
 0.5161772979585416
 0.41521947283013527
 0.10996610286788916
 1.0365392233362005

  0.000014 seconds (3 allocations: 384 bytes)
  0.000012 seconds (34 allocations: 3.031 KiB)


In [65]:
A = Float64.([10 1 2
    1 10 2
    1 1 5])
b = Float64.([13; 13; 7])
display(@time A \ b)
display(A * (A \ b))
display(@time gauss(size(A, 1), A, b))
# display(@time gauss(size(A, 1), A, b, false))  # implicit=false
# display(@time gauss(size(A, 1), A, b, true))  # implicit=true

3-element Vector{Float64}:
 1.0
 0.9999999999999998
 1.0

3-element Vector{Float64}:
 13.0
 12.999999999999998
  7.0

3-element Vector{Float64}:
 1.0
 0.9999999999999998
 1.0000000000000002

  0.000013 seconds (3 allocations: 288 bytes)
  0.000012 seconds (19 allocations: 1.453 KiB)


In [66]:
A = Float64.([4 -2 -4
    -2 17 10
    -4 10 9])
b = Float64.([-2; 25; 15])
display(@time A \ b)
# display(A * (A \ b))
display(@time gauss(size(A, 1), A, b))

3-element Vector{Float64}:
 1.0
 1.0
 1.0

3-element Vector{Float64}:
 1.0
 1.0
 1.0

  0.000015 seconds (3 allocations: 288 bytes)
  0.000014 seconds (19 allocations: 1.453 KiB)
